In [10]:
%run "../00_project_config.ipynb"

import pathlib as pl
import re as re
import collections as col
import pandas as pd
import datetime as dt

data_status_file = PROJECT_BASE.joinpath(
    "annotations", "projectmng", "data_production_status.tsv"
)

data_status_ts = None
with open(data_status_file, "r") as data_status_content:
    for line in data_status_content:
        if line.startswith("#"):
            if "2023" in line:
                print(f"Processing data status: {line.strip()}")
                data_status_ts = line.strip("#").strip()
            continue
        break
        
data_status = pd.read_csv(
    data_status_file, sep="\t", header=0,
    comment="#", index_col=1
)

# by construction, the fofn table contains only complete samples
fofn_table_file = PROJECT_BASE.joinpath(
    "samples", "fofn_table.tsv"
)

fofn_table = pd.read_csv(fofn_table_file, sep="\t", header=0, comment="#")

sample_records = col.defaultdict(dict)
for (sample, read_type), fofn_files in fofn_table.groupby(["sample", "read_type"]):
    
    try:
        sample_batch = int(data_status.at[sample, "sample_batch"])
        norm_name = sample
    except KeyError:
        norm_name = sample
        lu_name = "GM" + sample[2:]
        sample_batch = int(data_status.at[lu_name, "sample_batch"])
    merged_fofns = ",".join(sorted(fofn_files["fofn_path"].values))
    sample_records[norm_name][read_type] = merged_fofns
    sample_records[norm_name]["target"] = "unphased"
    sample_records[norm_name]["batch"] = sample_batch
    
sample_records = pd.DataFrame.from_records(sample_records)
sample_records = sample_records.transpose()
sample_records.reset_index(drop=False, inplace=True)
sample_records.rename({"index": "sample"}, inplace=True, axis=1)

sample_records.sort_values(["sample", "batch", "hifi", "ont"], inplace=True)
sample_records = sample_records[["sample", "batch", "target", "hifi", "ont"]]

sample_sheet_file = PROJECT_BASE.joinpath(
    "samples", "vrk_unps_samples.tsv"
)

with open(sample_sheet_file, "w") as sample_sheet:
    if data_status_ts is not None:
        _ = sample_sheet.write(f"# Relative to data status: {data_status_ts}\n")
    _ = sample_sheet.write(f"# {TODAY}\n")
    _ = sample_sheet.write(f"# Samples: {sample_records['sample'].nunique()}\n")
    sample_records.to_csv(sample_sheet, sep="\t", header=True, index=False)

#sample_records[["sample", "target", "hifi", "ont"]].to_csv(row_store, sep="\t", header=True, index=False)

# df = pd.DataFrame.from_records(sorted(records), columns=["sample", "key", "value"])
# df.to_csv(eav_store, header=True, index=False, sep="\t")

# def make_csv(values):
#     return ",".join(sorted(map(str, values)))

# df = df.pivot_table(
#     index="sample",
#     columns="key",
#     values="value",
#     aggfunc=make_csv
# )
# df = df[["hgsvc_sample_num", "target", "hifi", "ont", "batch"]]
# df.to_csv(row_store, header=True, index=True, sep="\t")


Processing data status: # 20230822T1023
